<div style="color:red;background-color:black">
Diamond Light Source
<br style="color:red;background-color:antiquewhite"><h1>Python Libraries: Nexus Files</h1>  

©2000-20 Chris Seddon 
</div>

## 1
At Diamond we often need to fit experimental data.  Python offers support with several libraries:
* numpy
* scipy
* matplotlib
* hdf5  

At Diamond, experimental data if often in the form of Nexus files - a number of samples Nexus files are used in this tutorial.  If you want to understand more about the structure of Nexus files, take a look at:  

* <a href="https://manual.nexusformat.org/">User Manual and Reference Documentation</a>

The Nexus files for this tutorial are stored in the "nexus" sub-directory:

In [ ]:
%%bash
ls data/nexus

## 2
Before analysing our Nexus files, be aware that you can also use the excellent "Dawn" workbench at Diamond Light Source.  Dawn is based on Eclipse and will automatically draw plots for Nexus files.  What we aim to do here is show how to produce equivalent plots in Jupyter notebook.  

Recall that to run Dawn you need to enter (at the command prompt):
<pre>module load dawn
dawn</pre>

Back in Jupyter notebook, we will start with the file <pre>data/nexus/i22-4996.nxs</pre>
Before we plot anything we need to know what keys are stored in this file.  If you are using Dawn, this information is automatically displayed, but in notebook we need to do a little work:

In [ ]:
import os
import h5py

f = h5py.File("data/nexus/i22-4996.nxs", "r")

for group in f:
    print(f[f"/{group}"])
    for subgroup in f[group]:
        print(f[f"/{group}/{subgroup}"])

## 3
We intend to plot the "Hotwaxs" data, so we need to establish how much data we have and the dimensional information for the data. This is easily achieved using:

In [ ]:
import h5py

f = h5py.File("data/nexus/i22-4996.nxs", "r")

# get dataset
ds = f["/entry1/Hotwaxs/data"]

# check shape (1,1,512)
print(ds.shape)

## 4
The nexus data needs to be converted to a Numpy array to be plotted.  In the above example the first two dimensions contain *DON'T KNOW* and need to be removed.  This will reduce the data to a one dimensional set which is easily plotted.  

Note that to convert a dataset to a Numpy array the expression
<pre>ds.value[0,0]</pre>
used to be used, but is now deprecated.  Instead we use
<pre>ds[(0,0)]</pre>

In [ ]:
%matplotlib inline
import h5py
import numpy as np
import matplotlib.pyplot as plt

# open Nexus file
f = h5py.File("data/nexus/i22-4996.nxs", "r")

# get dataset
ds = f["/entry1/Hotwaxs/data"]

# check shape (1,1,512)
print(f"dataset shape = {ds.shape}")

# extract last dimension to numpy array
X = ds[(0,0)]
size = X.shape[0] 
Y = np.arange(size)
ax = plt.gca()
ax.set_title("i22-4996.nxs")
ax.set_xlabel("X")
ax.set_ylabel("Hotwaxs")
plt.plot(Y, X)
plt.show()

## 5
Now we can plot the data independent of Dawn, we can perform additional customised processing as desired.  Say, for example, we wish to apply a filter to the data.  We could use SciPy's "savgol" filter:
* <a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.savgol_filter.html">scipy.signal.savgol_filter</a>

In [ ]:
%matplotlib inline
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter


# open Nexus file
f = h5py.File("data/nexus/i22-4996.nxs", "r")

# get dataset
ds = f["/entry1/Hotwaxs/data"]

# check shape (1,1,512)
print(f"dataset shape = {ds.shape}")

# extract last dimension to numpy array
Y = ds[(0,0)]
size = Y.shape[0] 
X = np.arange(size)
# window size 51, polynomial order 3
Y_filtered = savgol_filter(Y, 51, 3) 


ax = plt.gca()
ax.set_title("i22-4996.nxs")
ax.set_xlabel("X")
ax.set_ylabel("Hotwaxs")
plt.plot(X, Y)
plt.plot(X, Y_filtered)
plt.show()

## 6
Let's turn our attention to a different example; a 2 dimensional plot of data from
<pre>MoKedge_1_15.nxs</pre>

We will be making two plots: plotting the quantity "It" and differences "d(It)" against "energy".  To begin with we will open the Nexus file and check on the dimensionality of the data:

In [ ]:
%matplotlib inline
import h5py
import numpy as np
import matplotlib.pyplot as plt

fileName = "data/nexus/MoKedge_1_15.nxs"
f = h5py.File(fileName, "r")

X = f["/entry1/counterTimer01/Energy"][()]
Y = f["/entry1/counterTimer01/It"][()]

print(f"Shape of Energy data: {X.shape}")
print(f"Shape of It data: {Y.shape}")

## 7
When we take the differences of the "It" data we will end up with one less value.  We will need to take this into account in the subsequent plot by dropping one of the "Energy" points: 

In [ ]:
import numpy as np
dY = np.diff(Y)
print(f"Shape of d(It) data: {dY.shape}")

## 8
Let's take a look at the first plot:

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

fileName = "data/nexus/MoKedge_1_15.nxs"
f = h5py.File(fileName, "r")

Y = f["/entry1/counterTimer01/It"][()]
X = f["/entry1/counterTimer01/Energy"][()]
kwargs = {'linewidth':1, 'color':'red'}

# plot It
plt.plot(X, Y, **kwargs)
plt.show()

## 9
Adding some plot labels and including a Savgol filter we could improve the above with the code below.  Notice the use of Unicode variable names
<pre>Ŷ = savgol_filter ...
k̂wargs = ...</pre>

The Ŷ and k̂ characters are examples of combination Unicode characters.  

For more information on Unicode:
<a href="https://unicode-table.com/en">unicode table</a>

In [ ]:
%matplotlib inline
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter  # for smoothing

def set_title(title):
    figure = plt.gcf()
    figure.canvas.set_window_title(title)

def setupAxes(xLabel, yLabel):
    set_title("MoKedge_1_15.nxs")
    ax = plt.gca()
    ax.set_facecolor((1.0, 0.9, 0.6))
    ax.set_xlabel(xLabel)
    ax.set_ylabel(yLabel)
    ax.xaxis.set_label_position("top")
    ax.yaxis.set_label_position("right")
    ax.ticklabel_format(axis='y', style='sci', scilimits=(0,5))
    plt.grid(True)

fileName = "data/nexus/MoKedge_1_15.nxs"
f = h5py.File(fileName, "r")

Y = f["/entry1/counterTimer01/It"][()]
X = f["/entry1/counterTimer01/Energy"][()]
Ŷ = savgol_filter(Y, 25, 3) # smoothing window size 25, polynomial order 3

k̂wargs = {'linewidth':3, 'color':'blue'}
kwargs = {'linewidth':1, 'color':'red'}

# plot It
setupAxes('Energy', 'It')
plt.plot(X, Ŷ, **k̂wargs)
plt.plot(X, Y, **kwargs)
plt.show()

## 10
We can now add the differences plot.  Both plots are shown below:

In [ ]:
%matplotlib inline
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter  # for smoothing

def set_title(title):
    figure = plt.gcf()
    figure.canvas.set_window_title(title)

def setupAxes(xLabel, yLabel):
    set_title("MoKedge_1_15.nxs")
    ax = plt.gca()
    ax.set_facecolor((1.0, 0.9, 0.6))
    ax.set_xlabel(xLabel)
    ax.set_ylabel(yLabel)
    ax.xaxis.set_label_position("top")
    ax.yaxis.set_label_position("right")
    ax.ticklabel_format(axis='y', style='sci', scilimits=(0,5))
    plt.grid(True)

fileName = "data/nexus/MoKedge_1_15.nxs"
f = h5py.File(fileName, "r")

Y = f["/entry1/counterTimer01/It"][()]
X = f["/entry1/counterTimer01/Energy"][()]
Ŷ = savgol_filter(Y, 25, 3) # smoothing window size 25, polynomial order 3
dY = np.diff(Y)
dŶ = savgol_filter(dY, 25, 3) # smoothing window size 25, polynomial order 3

k̂wargs = {'linewidth':3, 'color':'blue'}
kwargs = {'linewidth':1, 'color':'red'}

# plot It
setupAxes('Energy', 'It')
plt.plot(X, Ŷ, **k̂wargs)
plt.plot(X, Y, **kwargs)
plt.show()

# plot d(It)
setupAxes('Energy', 'd(It)')
left = 280
right = 70
plt.plot(X[left:-right-1], dŶ[left:-right], **k̂wargs)
plt.plot(X[left:-right-1], dY[left:-right], **kwargs)
plt.show()